In [1]:
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")

In [95]:
text = 'वह मेरा भांजा है।'
text2 = 'He is my nephew.'

def align(text, text2):
    tokens = tokenizer(text, return_tensors='pt')
    tokens2 = tokenizer(text2, return_tensors='pt')
    left = model(**tokens)[0][0][1:-1]
    right = model(**tokens2)[0][0][1:-1]

    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    S = [[0 for i in range(len(right))] for j in range(len(left))]
    A, M = [[0 for i in range(len(right))] for j in range(len(left))], [[0 for i in range(len(right))] for j in range(len(left))]
    for pos_i, i in enumerate(left):
        for pos_j, j in enumerate(right):
            S[pos_i][pos_j] = float(cos(i, j))
            
    for _ in range(3):
        for i in range(len(left)):
            for j in range(len(right)):
                if max(sum([x[j] for x in A]), sum(A[i])) == 0:
                    M[i][j] = 1
                elif min(sum([x[j] for x in A]), sum(A[i])) > 0:
                    M[i][j] = 0
                else:
                    M[i][j] = 0.5
        
        maxes_i = [[0, -1]] * len(left)
        maxes_j = [[0, -1]] * len(right)
        for i in range(len(left)):
            for j in range(len(right)):
                val = S[i][j] * M[i][j]
                if val > maxes_i[i][0]: maxes_i[i] = [val, j]
                if val > maxes_j[j][0]: maxes_j[j] = [val, i]
        
        for i in range(len(left)):
            if maxes_j[maxes_i[i][1]][1] == i:
                A[i][maxes_i[i][1]] += 1
    
    print(text)
    print(text2)
    for pos_i, i in enumerate(A):
        for pos_j, j in enumerate(i):
            if j == 1:
                print(tokenizer.decode(tokens.input_ids[0][pos_i + 1:pos_i + 2]), tokenizer.decode(tokens2.input_ids[0][pos_j + 1:pos_j + 2]))

align(text, text2)

वह मेरा भांजा है।
He is my nephew.
वह He
मे my
भ nephew
है is
। .


In [183]:
def conv(x):
    x = x.lower()
    m = {
        'a': 'o', 'i': 'u', 'ī': 'ū', 'u': 'e', 'ā': 'ɔ', 'ɔ': 'a', 'o': 'i', 'e': 'ā',
        'k': 'b', 'g': 'p', 'ṅ': 'm',
        'c': 'j', 'j': 'c', 'ñ': 'ñ',
        't': 'ṭ', 'd': 'ḍ', 'n': 'ṇ',
        'p': 'k', 'b': 'g', 'm': 'ṅ',
        'ṭ': 't', 'ḍ': 'd', 'ṇ': 'n',
        'h': 'h',
        'r': 'l', 'l': 'r', 'ṛ': 'ḷ',
        '̤': '̤',
        's': 'ś', 'ś': 's',
        'v': 'z', 'z': 'v',
        'f': 'f',
        'y': 'w',
        'w': 'y',
        '\t': '\t',
        'ʰ': 'ʰ',
        ' ': ' '
    }
    return ''.join([m[y] if y in m else y for y in x])



In [184]:
print(conv("""śurkiu Nilufar ke gazzi genavɔn"""))

selbue ṇurefol bā povvu pāṇozaṇ
